In [9]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf
from datetime import datetime
from datetime import timedelta

In [10]:
def fetch_stocks():
    filters = ['ind_exchangetradedfund','sh_avgvol_o2000']
    # filters = ['sh_avgvol_o2000', 'ind_stocksonly', 'cap_largeover', 'sh_price_o10']
    
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [11]:
def cal_VRRS(ticker, ref, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * 100

In [12]:
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max()
    lowl = low.rolling(lookback).min()
    wr = 100 * ((close - highh) / (highh - lowl))
    return wr

In [13]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        if len(x) > 1:
            return float(x.replace('B', '')) * 1000000000
        return 1000000000.0
    if 'T' in x:
        if len(x) > 1:
            return float(x.replace('T', '')) * 1000000000000
        return 1000000000000000.0

    return 0.0

In [14]:
ticker_dict = {}

In [15]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  2.92it/s]


In [ ]:
start_date = "2017-01-01"
end_date = "2022-07-09"

interval_time = '1d'

# end_date = datetime.today().date()

spy = yf.Ticker("SPY")
spy = spy.history(start=start_date, end=end_date, interval=interval_time)['Close'].dropna()

for stock in stock_list:
    
    ticker = stock['Ticker']
    
    # avg_volume = value_to_float(stock['Avg Volume'])
    # if avg_volume > 2000000:

    stock = yf.Ticker(ticker)    

    stock_close = stock.history(start=start_date, end=end_date, interval=interval_time)['Close'].dropna()
    stock_now = stock.history(start=start_date, end=end_date, interval=interval_time).dropna()
    
    williams = get_wr(stock_now['High'], stock_now['Low'], stock_now['Close'], 2).iloc[-1]

    # Calculate the VRRS indicator:
    rolling_length = 21 

    vrrs = cal_VRRS(stock_close, spy, rolling_length).iloc[-1]
    previous_VRRS = cal_VRRS(stock_close.iloc[:-1], spy.iloc[:-1], rolling_length).iloc[-1]

    difference = vrrs - week_ago_VRRS
    difference_two = previous_VRRS - vrrs

    if  williams >= -30 and difference > 5:
        # print(ticker)
        print(ticker, ' || VRRS: ', round(vrrs, 2), '|| Previous VRRS: ', round(previous_VRRS, 2), ' || ', round(difference, 2))
    elif  williams <= -70 and difference_two > 5:
        # print(ticker)
        print(ticker, ' || VRRS: ', round(vrrs, 2), '|| Previous VRRS: ', round(previous_VRRS, 2), ' || ', round(difference, 2))

LABU  || VRRS:  44.7 || Previous VRRS:  42.68  ||  45.75
SVIX  || VRRS:  4.84 || Previous VRRS:  2.87  ||  5.89
SCO  || VRRS:  6.54 || Previous VRRS:  14.09  ||  7.58
TQQQ  || VRRS:  6.69 || Previous VRRS:  5.34  ||  7.74
TECL  || VRRS:  4.71 || Previous VRRS:  3.35  ||  5.75
ARKG  || VRRS:  14.89 || Previous VRRS:  16.52  ||  15.94
QLD  || VRRS:  4.4 || Previous VRRS:  3.69  ||  5.45
ARKK  || VRRS:  9.83 || Previous VRRS:  10.26  ||  10.88
